In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
import traceback

from LSTM_for_Stock.model import SequentialModel
from LSTM_for_Stock.data_processor import DataHelper
from LSTM_for_Stock.data_processor import DataLoaderStock
import logging
from LSTM_for_Stock.data_processor import Wrapper
from LSTM_for_Stock.data_processor import Wrapper_fillna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM,CuDNNLSTM
from keras.backend import clear_session
from keras.callbacks import EarlyStopping

import time
import numpy as np

class wrapper(Wrapper):
    def build(self, df):
        result = df.copy()
        result = result.fillna(method='ffill')
        result = result.drop(columns=['up_count', 'down_count'])
        return result.dropna()
    
class normalize(object):
    """数据标准化器"""

    def __init__(self, *args, **kwargs):
        pass

    def build(self, df):
        """执行数据标准化。**数据归一化**。

        Args:
            df (pd.DataFrame 或 pd.Series): 待处理的数据。

        Returns:
            pd.DataFrame 或 pd.Series: 与传入类型一致。
        """
        return np.round(df / df.iloc[0],8)

Using TensorFlow backend.


In [2]:
code='000002'
window=5
days=3
dl = DataLoaderStock(code, wrapper=wrapper())
df = dl.load()
train, test = DataHelper.train_test_split(df, batch_size=window + days)

X_train, Y_train = DataHelper.xy_split_2(train, window, days,norm=normalize())
X_test, Y_test = DataHelper.xy_split_2(test, window, days,norm=normalize())

X_train_arr = []
Y_train_arr = []
for x in X_train:
    X_train_arr.append(x.values)
for y in Y_train:
    Y_train_arr.append(y.values)
X_test_arr = []
Y_test_arr = []
for x in X_test:
    X_test_arr.append(x.values)
for y in Y_test:
    Y_test_arr.append(y.values)

In [3]:
clear_session()
model = Sequential()
model.add(CuDNNLSTM(128, input_shape=X_train_arr[0].shape, return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(64))
model.add(Dropout(0.2))
model.add(Dense(days))
model.compile(loss='mse',
             optimizer="rmsprop",
             metrics=["mae", "acc"])
start = time.time()
history = model.fit(
    np.array(X_train_arr),
    np.array(Y_train_arr),
    epochs=5,
    verbose=2,
    batch_size=128,
    validation_split=0.15
)
end = time.time()
print('Net time using CuDNNLSTM : ', end-start, ' secs.')

Train on 2497 samples, validate on 441 samples
Epoch 1/5
 - 7s - loss: 0.0964 - mean_absolute_error: 0.2138 - acc: 0.3332 - val_loss: 0.0215 - val_mean_absolute_error: 0.1281 - val_acc: 0.2336
Epoch 2/5
 - 1s - loss: 0.0273 - mean_absolute_error: 0.1323 - acc: 0.3236 - val_loss: 0.0086 - val_mean_absolute_error: 0.0704 - val_acc: 0.5782
Epoch 3/5
 - 1s - loss: 0.0286 - mean_absolute_error: 0.1352 - acc: 0.3400 - val_loss: 0.0066 - val_mean_absolute_error: 0.0555 - val_acc: 0.5442
Epoch 4/5
 - 1s - loss: 0.0227 - mean_absolute_error: 0.1208 - acc: 0.3356 - val_loss: 0.0052 - val_mean_absolute_error: 0.0477 - val_acc: 0.1791
Epoch 5/5
 - 1s - loss: 0.0233 - mean_absolute_error: 0.1203 - acc: 0.3224 - val_loss: 0.0052 - val_mean_absolute_error: 0.0474 - val_acc: 0.2381
Net time using CuDNNLSTM :  10.926363229751587  secs.
